In [13]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedKFold

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

# import matplotlib.pyplot as plt

In [14]:
def apply_smote(df):
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [50]:
def get_statistics(project,metric):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    understand_df['Name'] = understand_df.Name.str.rsplit('.',1).str[1]
    
    commit_guru_file_level_path = 'data/commit_guru_file/' + project + '.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_file_level_df = commit_guru_file_level_df[commit_guru_file_level_df['file_name'].str.contains('.java')]
    commit_guru_file_level_df['Name'] = commit_guru_file_level_df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')
    commit_guru_file_level_df = commit_guru_file_level_df.drop('file_name',axis = 1)
    
    
    df = understand_df.merge(commit_guru_file_level_df,how='left',on=['commit_hash','Name'])
    
    
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    df = df[cols]
    
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
#     df.dropna(inplace=True)
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
#     scaler = MinMaxScaler()
#     X = scaler.fit_transform(X)
#     X = pd.DataFrame(X,columns = cols)
    imp_mean = IterativeImputer(random_state=0)
    X = imp_mean.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    X = pd.DataFrame(X.quantile([.25, .5, .75]))
    X['project'] = [project]*X.shape[0]
    X['quantile'] = [.25, .5, .75]
    return X

In [51]:
def run_self(project,metric):
    X = get_statistics(project,metric)
    return X

In [52]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [55]:
final_df = pd.DataFrame()
for project in projects[150:]:
    try:
        if project == '.DS_Store':
            continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        x = run_self(project,'process')
        final_df = pd.concat([final_df,x])
        print(final_df)
    except Exception as e:
        print(e)
        continue

+++++++++++++++++   spring-xd-samples  +++++++++++++++++
Shape of passed values is (62, 45), indices imply (62, 66)
+++++++++++++++++   environment-dashboard  +++++++++++++++++
      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   

      AvgLine  AvgLineBlank  AvgLineCode  AvgLineComment  CountClassBase  \
0.25     7.00           0.0          6.0             0.0             1.0   
0.50    15.50           0.0         11.0             0.0             1.0   
0.75    19.75           1.0         16.0             0.0             1.0   

      CountClassCoupled  ...  file_avg_nadev  file_avg_ncomm  file_ns  \
0.25                0.0  ...        0.000000        0.000000      1.0   
0.50                1.0  ...        2.692877   

      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    2.0                  3.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            5.0         

      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    2.0                  3.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            5.0         

      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    2.0                  3.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            5.0         

      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    2.0                  3.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            5.0         

...

      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
...             ...                    ...                  ...           ...   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
0.75            1.0                    1.0                  1.0           1.0   

      AvgLine  AvgLineBlank

      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
...             ...                    ...                  ...           ...   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   

      AvgLine  AvgLineBlank

      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
...             ...                    ...                  ...           ...   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
0.75            1.0                    1.0                  1.0           1.0   

      AvgLine  AvgLineBlank

      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
...             ...                    ...                  ...           ...   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   

      AvgLine  AvgLineBlank

      AvgCyclomatic  AvgCyclomaticModified  AvgCyclomaticStrict  AvgEssential  \
0.25            1.0                    1.0                  1.0           1.0   
0.50            2.0                    2.0                  2.0           1.0   
0.75            3.0                    3.0                  3.0           2.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
...             ...                    ...                  ...           ...   
0.50            1.0                    1.0                  1.0           1.0   
0.75            1.0                    1.0                  1.0           1.0   
0.25            1.0                    1.0                  1.0           1.0   
0.50            1.0                    1.0                  1.0           1.0   
0.75            2.0                    2.0                  2.0           1.0   

      AvgLine  AvgLineBlank

In [12]:
# with open('results/Performance/commit_guru_file_specific/process_700_svc_5_fold_5_repeat.pkl', 'wb') as handle:
#     pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [57]:
# final_df.to_csv('results/metric_statistics_all.csv')

In [68]:
median_df = final_df[final_df['quantile'] == 0.75]

In [69]:
median_df.to_csv('results/metric_statistics_75.csv')